In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

'3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:55:37) \n[Clang 14.0.6 ]'

In [3]:
%load_ext autoreload 
%autoreload 2

### Train Test split

In [4]:
org_df = pd.read_excel("../../data/Hackathon Data.xlsx")
org_df.shape

(846632, 11)

In [5]:
org_df["Encoded_SKU_ID"].nunique()

575

In [6]:
org_df["SALES_DATE"] = pd.to_datetime(org_df["SALES_DATE"])

In [7]:
end_date = org_df.groupby(["Encoded_SKU_ID"])["SALES_DATE"].max().reset_index()
end_date.columns = ["Encoded_SKU_ID","max_SALES_DATE"]
end_date.head()

,Encoded_SKU_ID,max_SALES_DATE
0,1,2022-07-31
1,2,2022-07-31
2,3,2022-07-31
3,4,2022-07-31
4,5,2022-07-31


In [8]:
end_date["max_SALES_DATE"].value_counts()

2022-07-31    556
2022-07-30     14
2022-07-23      2
2022-07-09      1
2022-06-25      1
2021-11-20      1
Name: max_SALES_DATE, dtype: int64

In [9]:
train_test = org_df.merge(end_date,on="Encoded_SKU_ID",how="left")

In [10]:
train_test["delta_days"] = (train_test["max_SALES_DATE"]-train_test["SALES_DATE"]).apply(lambda x: x.days)

In [11]:
train_test["validation"] = pd.to_numeric(train_test["delta_days"]<7)
train_test.groupby(["validation"]).agg(
{
    "Encoded_SKU_ID":[len,lambda x: x.nunique()]
})

Encoded_SKU_ID           
                      len <lambda_0>
validation                          
False              842673        575
True                 3959        575

In [12]:
check = train_test.groupby("Encoded_SKU_ID")["validation"].sum()
check.value_counts()

7    564
1     11
Name: validation, dtype: int64

In [13]:
a = end_date[end_date["max_SALES_DATE"]!="2022-07-31"]["Encoded_SKU_ID"].unique()

In [14]:
b = check[check!=7].index

In [15]:
len(set(a).union(set(b)))

30

In [16]:
train_test["validation_clean"] = pd.to_numeric((train_test["max_SALES_DATE"]=="2022-07-31") &
                                              (~train_test["Encoded_SKU_ID"].isin(check[check!=7].index)))

In [17]:
train_test.groupby(["validation_clean"]).agg(
{
    "Encoded_SKU_ID":[len,lambda x: x.nunique()]
})

Encoded_SKU_ID           
                            len <lambda_0>
validation_clean                          
False                     45327         30
True                     801305        545

In [18]:
train_test.groupby(["validation","validation_clean"]).agg(
{
    "Encoded_SKU_ID":[len,lambda x: x.nunique()]
})

Encoded_SKU_ID           
                                       len <lambda_0>
validation validation_clean                          
False      False                     45183         30
           True                     797490        545
True       False                       144         30
           True                       3815        545

In [19]:
train_test.to_csv("../../data/train_validation_marker.csv", index = False)